In [1]:
from google.colab import drive
drive.mount('/content/drive/')

!cp "/content/drive/My Drive/Dissertation/envs/point_maze.py" .

Mounted at /content/drive/


In [2]:
# for inference, not continued training
def save_model(model, name):
    path = f"/content/drive/My Drive/Dissertation/saved_models/point_maze_time/{name}" 

    torch.save({
      'controller': {
          'critic': model.critic.state_dict(),
          'actor': model.actor.state_dict(),
      }
    }, path)

import copy
def load_model(model, name):
    path = f"/content/drive/My Drive/Dissertation/saved_models/point_maze_time/{name}" 
    checkpoint = torch.load(path)

    model.critic.load_state_dict(checkpoint['controller']['critic'])
    model.critic_target = copy.deepcopy(model.critic)
    
    model.actor.load_state_dict(checkpoint['controller']['actor'])
    model.actor_target = copy.deepcopy(model.actor)

    model.eval()

In [3]:
%matplotlib inline

import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

from IPython import display
plt.ion()

# if gpu is to be used
device = torch.device("cuda")

In [4]:
class NormalizedEnv(gym.ActionWrapper):
    """ Wrap action """

    def action(self, action):
        act_k = (self.action_space.high - self.action_space.low)/ 2.
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k * action + act_b

    def reverse_action(self, action):
        act_k_inv = 2./(self.action_space.high - self.action_space.low)
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k_inv * (action - act_b)

In [5]:
from point_maze import PointMazeEnv 
env = NormalizedEnv(PointMazeEnv(4))

***

In [6]:
def plot_durations(episode_durations, actions):
    fig, axs = plt.subplots(2, figsize=(10,10))
    
    durations_t, durations = list(map(list, zip(*episode_durations)))
    durations = torch.tensor(durations, dtype=torch.float)
    
    fig.suptitle('Training')
    axs[0].set_xlabel('Episode')
    axs[0].set_ylabel('Reward')
    
    axs[0].plot(durations_t, durations.numpy())

    durations_t, durations = list(map(list, zip(*actions)))
    durations = torch.tensor(durations, dtype=torch.float)
    axs[1].plot(durations_t, durations.numpy())
        
    plt.pause(0.001)  # pause a bit so that plots are updated
    display.clear_output(wait=True)

In [7]:
# [reference] https://github.com/matthiasplappert/keras-rl/blob/master/rl/random.py

class RandomProcess(object):
    def reset_states(self):
        pass

class AnnealedGaussianProcess(RandomProcess):
    def __init__(self, mu, sigma, sigma_min, n_steps_annealing):
        self.mu = mu
        self.sigma = sigma
        self.n_steps = 0

        if sigma_min is not None:
            self.m = -float(sigma - sigma_min) / float(n_steps_annealing)
            self.c = sigma
            self.sigma_min = sigma_min
        else:
            self.m = 0.
            self.c = sigma
            self.sigma_min = sigma

    @property
    def current_sigma(self):
        sigma = max(self.sigma_min, self.m * float(self.n_steps) + self.c)
        return sigma


# Based on http://math.stackexchange.com/questions/1287634/implementing-ornstein-uhlenbeck-in-matlab
class OrnsteinUhlenbeckProcess(AnnealedGaussianProcess):
    def __init__(self, theta, mu=0., sigma=1., dt=1e-2, x0=None, size=1, sigma_min=None, n_steps_annealing=1000):
        super(OrnsteinUhlenbeckProcess, self).__init__(mu=mu, sigma=sigma, sigma_min=sigma_min, n_steps_annealing=n_steps_annealing)
        self.theta = theta
        self.mu = mu
        self.dt = dt
        self.x0 = x0
        self.size = size
        self.reset_states()

    def sample(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + self.current_sigma * np.sqrt(self.dt) * np.random.normal(size=self.size)
        self.x_prev = x
        self.n_steps += 1
        return x

    def reset_states(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros(self.size)

In [8]:
def soft_update(target, source, tau):
    for target_param, param in zip(target.parameters(), source.parameters()):
        target_param.data.copy_(
            target_param.data * (1.0 - tau) + param.data * tau
        )

def hard_update(target, source):
    for target_param, param in zip(target.parameters(), source.parameters()):
            target_param.data.copy_(param.data)

In [9]:
# (state, action) -> (next_state, reward, done)
transition = namedtuple('transition', ('state', 'action', 'next_state', 'reward', 'done'))

# replay memory D with capacity N
class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    # implemented as a cyclical queue
    def store(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        
        self.memory[self.position] = transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

***

In [10]:
class Actor(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(nb_states, 128)
        self.fc2 = nn.Linear(128, 128)
        self.head = nn.Linear(128, nb_actions)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return torch.tanh(self.head(x))

class Critic(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(Critic, self).__init__()

        # Q1 architecture
        self.l1 = nn.Linear(nb_states + nb_actions, 128)
        self.l2 = nn.Linear(128, 128)
        self.l3 = nn.Linear(128, 1)

        # Q2 architecture
        self.l4 = nn.Linear(nb_states + nb_actions, 128)
        self.l5 = nn.Linear(128, 128)
        self.l6 = nn.Linear(128, 1)
    
    def forward(self, state, action):
        sa = torch.cat([state, action], 1).float()

        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)

        q2 = F.relu(self.l4(sa))
        q2 = F.relu(self.l5(q2))
        q2 = self.l6(q2)
        return q1, q2

    def Q1(self, state, action):
        sa = torch.cat([state, action], 1).float()

        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)
        return q1

In [11]:
BATCH_SIZE = 64
GAMMA = 0.99

# https://spinningup.openai.com/en/latest/algorithms/td3.html
class TD3(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(TD3, self).__init__()
        self.nb_states = nb_states
        self.nb_actions= nb_actions
        
        self.actor = Actor(self.nb_states, self.nb_actions)
        self.actor_target = Actor(self.nb_states, self.nb_actions)
        self.actor_optimizer  = optim.Adam(self.actor.parameters(), lr=0.0001)

        self.critic = Critic(self.nb_states, self.nb_actions)
        self.critic_target = Critic(self.nb_states, self.nb_actions)
        self.critic_optimizer  = optim.Adam(self.critic.parameters(), lr=0.0001)

        hard_update(self.actor_target, self.actor)
        hard_update(self.critic_target, self.critic)
        
        #Create replay buffer
        self.memory = ReplayMemory(100000)
        self.random_process = OrnsteinUhlenbeckProcess(size=nb_actions, theta=0.15, mu=0.0, sigma=0.2)

        # Hyper-parameters
        self.tau = 0.005
        self.depsilon = 1.0 / 50000
        self.policy_noise=0.2
        self.noise_clip=0.5
        self.policy_freq=2
        self.total_it = 0

        # 
        self.epsilon = 1.0
        self.is_training = True

    def update_policy(self):
        if len(self.memory) < BATCH_SIZE:
            return

        self.total_it += 1
        
        # in the form (state, action) -> (next_state, reward, done)
        transitions = self.memory.sample(BATCH_SIZE)
        batch = transition(*zip(*transitions))
        
        state_batch = torch.cat(batch.state)
        next_state_batch = torch.cat(batch.next_state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        done_mask = np.array(batch.done)
        not_done_mask = torch.from_numpy(1 - done_mask).float().to(device)

        # Target Policy Smoothing
        with torch.no_grad():
            # Select action according to policy and add clipped noise
            noise = (
                torch.randn_like(action_batch) * self.policy_noise
            ).clamp(-self.noise_clip, self.noise_clip).float()
            
            next_action = (
                self.actor_target(next_state_batch) + noise
            ).clamp(-1.0, 1.0).float()

            # Compute the target Q value
            # Clipped Double-Q Learning
            target_Q1, target_Q2 = self.critic_target(next_state_batch, next_action)
            target_Q = torch.min(target_Q1, target_Q2).squeeze(1)
            target_Q = (reward_batch + GAMMA * not_done_mask  * target_Q).float()
        
        # Critic update
        current_Q1, current_Q2 = self.critic(state_batch, action_batch)
      
        critic_loss = F.mse_loss(current_Q1, target_Q.unsqueeze(1)) + F.mse_loss(current_Q2, target_Q.unsqueeze(1))

        # Optimize the critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Delayed policy updates
        if self.total_it % self.policy_freq == 0:
            # Compute actor loss
            actor_loss = -self.critic.Q1(state_batch, self.actor(state_batch)).mean()
            
            # Optimize the actor 
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()

            # Target update
            soft_update(self.actor_target, self.actor, self.tau)
            soft_update(self.critic_target, self.critic, self.tau)

    def eval(self):
        self.actor.eval()
        self.actor_target.eval()
        self.critic.eval()
        self.critic_target.eval()

    def observe(self, s_t, a_t, s_t1, r_t, done):
        self.memory.store(s_t, a_t, s_t1, r_t, done)

    def random_action(self):
        return torch.tensor([np.random.uniform(-1.,1.,self.nb_actions)], device=device, dtype=torch.float)

    def select_action(self, s_t, warmup, decay_epsilon):
        if warmup:
            return self.random_action()

        with torch.no_grad():
            action = self.actor(s_t).squeeze(0)
            #action += torch.from_numpy(self.is_training * max(self.epsilon, 0) * self.random_process.sample()).to(device).float()
            action += torch.from_numpy(self.is_training * max(self.epsilon, 0) * np.random.uniform(-1.,1.,1)).to(device).float()
            action = torch.clamp(action, -1., 1.)

            action = action.unsqueeze(0)
            
            if decay_epsilon:
                self.epsilon -= self.depsilon
            
            return action

In [12]:
import time
SAVE_OFFSET = 10

def train_model():
    global SAVE_OFFSET

    n_observations = env.observation_space.shape[0]
    n_actions = env.action_space.shape[0]
    
    agent = TD3(n_observations, n_actions).to(device)
    
    max_episode_length = 500
    
    agent.is_training = True
    episode_reward = 0.
    observation = None
    
    warmup = 100
    num_episodes = 3000 # M
    episode_durations = []
    actions = [(0,0)]

    steps = 0
    for i_episode in range(num_episodes):
        observation = env.reset()
        state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
        
        overall_reward = 0
        episode_steps = 0
        done = False
        while not done:        
            # agent pick action ...
            action = agent.select_action(state, i_episode <= warmup, True)

            # env response with next_observation, reward, terminate_info
            action_i = action.detach().cpu().squeeze(0).numpy()

            observation, reward, done, info = env.step(action_i)
            steps += 1

            #actions.append((steps, action_i[0]))
                
            if max_episode_length and episode_steps >= max_episode_length - 1:
                done = True
            episode_steps += 1 
                
            extrinsic_reward = torch.tensor([reward], device=device)
            
            overall_reward += reward
            
            next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            # agent observe and update policy
            agent.observe(state, action, next_state, extrinsic_reward, done)           
            state = next_state
            
            if i_episode > warmup:
                agent.update_policy()

        episode_durations.append((i_episode, overall_reward))
        #plot_durations(episode_durations, actions)

        _, dur = list(map(list, zip(*episode_durations)))
        if len(dur) > 100:
            if i_episode % 100 == 0:
                print(f"{i_episode}: {np.mean(dur[-100:])}")
            if i_episode >= 300 and i_episode % 100 == 0 and np.mean(dur[-100:]) <= -49.0:
                print(f"Unlucky after {i_episode} eps! Terminating...")
                return None
            if np.mean(dur[-100:]) >= 90:
                print(f"Solved after {i_episode} episodes!")
                save_model(agent, f"td3_{SAVE_OFFSET}")
                SAVE_OFFSET += 1
                return agent
    return None

In [ ]:
i = 17
while i < 17:
    agent = train_model()
    
    if agent is not None:
        print(i)
        i += 1

In [ ]:
state_max = torch.from_numpy(env.observation_space.high).to(device).float()
state_min = torch.from_numpy(env.observation_space.low).to(device).float()
state_mid = (state_max + state_min) / 2.
state_range = (state_max - state_min)
def eval_model(agent, episode_durations):
    agent.eval()
    agent.is_training = False

    max_episode_length = 500
    num_episodes = 100

    for noise in np.arange(0,0.51,0.05):

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()
            # unsqueeze adds batch dimension
            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            episode_steps = 0
            done = False
            while not done:
                state = state + state_range * torch.FloatTensor(state.shape).uniform_(-noise, noise).to(device)
                state = torch.max(torch.min(state, state_max), state_min).float()

                action = agent.select_action(state, False, False)
                action_i = action.detach().cpu().squeeze(0).numpy()

                observation, reward, done, info = env.step(action_i) 
                
                if max_episode_length and episode_steps >= max_episode_length - 1:
                    done = True
                
                episode_steps += 1 

                overall_reward += reward

                state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_durations[np.round(noise, 2)].append(overall_reward / num_episodes)

In [ ]:
state_max = torch.from_numpy(env.observation_space.high).to(device).float()
state_min = torch.from_numpy(env.observation_space.low).to(device).float()
state_mid = (state_max + state_min) / 2.
state_range = (state_max - state_min)
def fgsm_attack(data, eps, data_grad):
    sign_data_grad = data_grad.sign()

    perturbed_data = data + eps * sign_data_grad * state_range

    clipped_perturbed_data = torch.max(torch.min(perturbed_data, state_max), state_min)

    return clipped_perturbed_data

def fgsm_action(state, agent, eps, target, targeted):
    state = state.clone().detach().requires_grad_(True)

    if targeted:
        # initial forward pass
        action = agent.actor(state)
        action = torch.clamp(action, -1., 1.)

        loss = F.mse_loss(action, target)
    else:
        loss = agent.critic.Q1(state, agent.actor(state)).mean()

    agent.actor.zero_grad()
    agent.critic.zero_grad()

    # calc loss
    loss.backward()
    data_grad = state.grad.data
    # perturb state
    state_p = fgsm_attack(state, eps, data_grad).float()
    return agent.select_action(state_p, False, False)

def apply_fgsm(agent, episode_durations, targeted):
    TARGET_ACTION = torch.tensor([[0.0, 0.0]], device=device, dtype=torch.float)

    agent.eval()

    max_episode_length = 500
    agent.is_training = False

    num_episodes = 100

    for eps in np.arange(0.0, 0.201, 0.02):

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()
            # unsqueeze adds batch dimension
            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            episode_steps = 0
            done = False
            while not done:
                action = fgsm_action(state, agent, eps, TARGET_ACTION, targeted)
                action_i = action.detach().cpu().squeeze(0).numpy()

                observation, reward, done, info = env.step(action_i)
                
                if max_episode_length and episode_steps >= max_episode_length - 1:
                    done = True
                
                episode_steps += 1 

                overall_reward += reward

                state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_durations[eps].append(overall_reward / num_episodes)

In [ ]:
def plot_norms(episode_durations):
    plt.figure(2, figsize=(10,10))
    
    x, ys = np.array(list(episode_durations.keys())), np.array(list(episode_durations.values()))
    
    plt.title('Action Prediction $\mu$ and $\pm \sigma$ interval')
    plt.xlabel('L2 Norm')
    plt.ylabel('Average Reward')
    
    mu = np.mean(ys, axis=1)
    plt.plot(x, mu)
    stds = np.std(ys, axis = 1)
    plt.fill_between(x, mu + stds , mu - stds, alpha=0.2)
        
    plt.pause(0.001)  # pause a bit so that plots are updated
    display.clear_output(wait=True)

In [ ]:
episodes = {}
for l2norm in np.arange(0,0.51,0.05):
    episodes[np.round(l2norm, 2)] = []

fgsm_t = {}
fgsm_ut = {}
for eps in np.arange(0.0, 0.201, 0.02):
    fgsm_t[eps] = []
    fgsm_ut[eps] = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

i = 0
while i < 10:
    #agent = train_model()
    agent = TD3(n_observations, n_actions).to(device)
    load_model(agent, f"td3_{i}")

    if agent is not None:
        #eval_model(agent, episodes)
        #apply_fgsm(agent, fgsm_t, True)
        apply_fgsm(agent, fgsm_ut, False)
        #print(f"{i} noise: {episodes}")
        #print(f"{i} fgsm (t): {fgsm_t}")
        print(f"{i} fgsm (ut): {fgsm_ut}")
        i += 1

print("---")
#print(f"noise: {episodes}")
#print(f"fgsm (t): {fgsm_t}")
print(f"fgsm (ut): {fgsm_ut}")

0 fgsm (ut): {0.0: [95.87099999999282], 0.02: [97.62799999999568], 0.04: [42.79100000000615], 0.06: [20.54100000001381], 0.08: [16.828000000000614], 0.1: [-31.282999999971825], 0.12: [-42.30899999997428], 0.14: [-50.00000000000659], 0.16: [-45.40499999998737], 0.18: [-45.43399999998747], 0.2: [-47.78399999999711]}
1 fgsm (ut): {0.0: [95.87099999999282, 97.76799999999587], 0.02: [97.62799999999568, 80.97399999998805], 0.04: [42.79100000000615, -23.00199999997912], 0.06: [20.54100000001381, -5.463000000006698], 0.08: [16.828000000000614, -21.93799999998347], 0.1: [-31.282999999971825, -1.848000000002547], 0.12: [-42.30899999997428, -7.750000000008004], 0.14: [-50.00000000000659, -26.161999999977898], 0.16: [-45.40499999998737, -17.730999999981762], 0.18: [-45.43399999998747, -33.53799999997343], 0.2: [-47.78399999999711, -18.815999999991664]}
2 fgsm (ut): {0.0: [95.87099999999282, 97.76799999999587, 92.32999999999089], 0.02: [97.62799999999568, 80.97399999998805, -42.69199999997886], 0.0

In [ ]:
print("fgsm (t): {0.0: [95.87499999999261, 97.77399999999591, 94.40799999999314, 96.49599999999695, 96.05199999999255, 93.26499999999466, 97.67499999999583, 96.19599999999672, 96.38699999999682, 97.90299999999614], 0.02: [79.72499999998196, 21.991000000009002, -3.1010000000024833, 41.113000000007496, 85.9739999999861, 9.863000000004849, -13.443999999999521, -35.91399999997348, 26.180000000020556, 54.89999999998844], 0.04: [73.9719999999884, -23.595999999986894, -23.778999999989086, 20.395000000016086, 35.00600000001057, 44.89400000000183, 14.32600000000494, -4.275000000004516, 53.81599999999942, 63.61199999999205], 0.06: [24.829000000017007, -45.642999999989144, -13.146999999987187, -2.91899999999977, 17.378000000015206, 29.93000000001121, -39.78299999997068, 2.9480000000004143, 41.63500000001305, 72.57899999998226], 0.08: [6.6309999999976705, -34.36599999997397, -8.729000000003882, -42.810999999986116, -25.77299999997859, 42.20100000001005, -47.15799999999459, -19.246999999983736, 61.10699999999787, 73.96399999998457], 0.1: [13.876000000002794, -43.61399999998562, -26.5579999999776, -37.31899999997468, -15.181000000000543, 32.47100000001505, -43.334999999978926, -37.311999999973004, 59.75999999999233, 61.70599999998308], 0.12: [29.658000000014457, -45.04099999998945, -13.630999999999093, -43.71199999998144, -12.977999999999417, 47.40900000000926, -26.055999999978848, -30.412999999977384, 51.740999999990784, 0.6499999999996606], 0.14: [-15.67999999999861, -48.97600000000195, -35.42999999997022, -32.84999999997244, -30.01599999997857, 20.647000000005374, -24.083999999978936, -40.279999999968815, 36.544000000019366, -22.462999999981506], 0.16: [-48.688000000000905, -45.91499999999012, -33.186999999973494, -38.71399999997188, -37.35399999997284, -1.7050000000007588, -27.89899999997665, -44.2919999999831, 24.02100000001258, -32.91799999997237], 0.18: [-48.56400000000045, -45.32499999999101, -35.79399999997332, -23.90199999998022, -23.642999999983036, -21.72199999998669, -30.350999999976317, -45.728999999989455, 2.421999999999141, -45.74699999999016], 0.2: [-50.00000000000659, -50.00000000000659, -46.65799999999266, -25.24599999997427, -9.83500000000743, -36.045999999972196, -38.621999999972644, -50.00000000000659, -39.09699999996903, -47.124999999995055]}")

fgsm (t): {0.0: [95.87499999999261, 97.77399999999591, 94.40799999999314, 96.49599999999695, 96.05199999999255, 93.26499999999466, 97.67499999999583, 96.19599999999672, 96.38699999999682, 97.90299999999614], 0.02: [79.72499999998196, 21.991000000009002, -3.1010000000024833, 41.113000000007496, 85.9739999999861, 9.863000000004849, -13.443999999999521, -35.91399999997348, 26.180000000020556, 54.89999999998844], 0.04: [73.9719999999884, -23.595999999986894, -23.778999999989086, 20.395000000016086, 35.00600000001057, 44.89400000000183, 14.32600000000494, -4.275000000004516, 53.81599999999942, 63.61199999999205], 0.06: [24.829000000017007, -45.642999999989144, -13.146999999987187, -2.91899999999977, 17.378000000015206, 29.93000000001121, -39.78299999997068, 2.9480000000004143, 41.63500000001305, 72.57899999998226], 0.08: [6.6309999999976705, -34.36599999997397, -8.729000000003882, -42.810999999986116, -25.77299999997859, 42.20100000001005, -47.15799999999459, -19.246999999983736, 61.1069999

In [ ]:
print("noise: {0.0: [95.84299999999237, 97.7699999999959, 94.83099999999098, 94.98799999999737, 96.63299999999461, 93.457999999995, 94.84499999999221, 90.14699999999331, 96.33899999999399, 97.90899999999614], 0.05: [81.38499999999286, 1.518999999999962, 25.856000000017605, 50.32099999999123, 41.773000000015244, 65.34799999999119, 2.906999999999274, -23.120999999979528, 58.1009999999882, 66.32899999998857], 0.1: [56.43899999999991, -7.056000000005732, 10.625000000001517, 14.278000000010868, 6.115999999997395, 61.41799999998866, -2.766000000006798, -13.371999999993127, 80.32399999998538, 56.840999999987986], 0.15: [44.458000000001604, 6.343999999995318, 44.91200000000099, 25.281000000013545, 6.320999999997017, 76.73299999998343, 30.108000000007845, 6.625000000001936, 87.63199999999286, 75.76499999998076], 0.2: [35.54700000001017, 49.008000000004074, 18.23200000002292, 57.43399999997138, -11.933000000001636, 70.42399999999164, 65.46499999997067, 14.3760000000066, 78.03199999998415, 74.47799999998215], 0.25: [29.995000000020138, 54.97999999997475, 10.650999999994085, 61.34699999997909, -30.576999999973562, 65.66899999997031, 72.30299999996966, 15.57100000000391, 63.02999999997938, 62.241999999967426], 0.3: [17.133000000009297, 42.36000000002272, 3.421999999994482, 66.00399999998352, -37.381999999971896, 63.350999999968536, 71.97299999997163, 8.074999999998646, 25.287000000012924, 27.91300000001797], 0.35: [17.181000000006282, 35.287000000022665, -7.517000000002699, 74.87299999997555, -42.91299999997626, 55.74399999999955, 68.34099999997001, 17.96100000001002, -0.6950000000019384, 15.565999999996926], 0.4: [17.51000000001047, 26.875000000016986, -8.581000000004389, 76.24399999997742, -33.78999999997332, 50.316000000003186, 44.792000000018014, -3.6930000000017897, -21.92899999997987, 15.904000000008962], 0.45: [-7.397000000008017, 18.74100000001584, -10.991999999998422, 67.11699999999045, -34.330999999969706, 41.14800000001356, 14.901000000005427, 1.891999999995767, -29.359999999979426, -8.438000000003507], 0.5: [-6.232000000004591, 16.731000000005352, -19.90699999998307, 67.55499999998452, -32.70099999997604, 46.18500000001505, -2.5990000000007227, 8.785999999996113, -28.49299999997531, 1.7389999999947081]}")

noise: {0.0: [95.84299999999237, 97.7699999999959, 94.83099999999098, 94.98799999999737, 96.63299999999461, 93.457999999995, 94.84499999999221, 90.14699999999331, 96.33899999999399, 97.90899999999614], 0.05: [81.38499999999286, 1.518999999999962, 25.856000000017605, 50.32099999999123, 41.773000000015244, 65.34799999999119, 2.906999999999274, -23.120999999979528, 58.1009999999882, 66.32899999998857], 0.1: [56.43899999999991, -7.056000000005732, 10.625000000001517, 14.278000000010868, 6.115999999997395, 61.41799999998866, -2.766000000006798, -13.371999999993127, 80.32399999998538, 56.840999999987986], 0.15: [44.458000000001604, 6.343999999995318, 44.91200000000099, 25.281000000013545, 6.320999999997017, 76.73299999998343, 30.108000000007845, 6.625000000001936, 87.63199999999286, 75.76499999998076], 0.2: [35.54700000001017, 49.008000000004074, 18.23200000002292, 57.43399999997138, -11.933000000001636, 70.42399999999164, 65.46499999997067, 14.3760000000066, 78.03199999998415, 74.4779999999

In [ ]:
def eval_scale(agent, episode_durations):
    agent.eval()
    agent.is_training = False

    max_episode_length = 500
    num_episodes = 100

    for scale in np.arange(1.0,7.01,0.5):
        env = NormalizedEnv(PointMazeEnv(scale))

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()
            # unsqueeze adds batch dimension
            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            episode_steps = 0
            done = False
            while not done:

                action = agent.select_action(state, False, False)
                action_i = action.detach().cpu().squeeze(0).numpy()

                observation, reward, done, info = env.step(action_i) 
                
                if max_episode_length and episode_steps >= max_episode_length - 1:
                    done = True
                
                episode_steps += 1 

                overall_reward += reward

                state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_durations[np.round(scale, 2)].append(overall_reward / num_episodes)

In [ ]:
episodes = {}
for scale in np.arange(1.0,7.01,0.5):
    episodes[np.round(scale, 2)] = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

i = 0
while i < 10:
    agent = TD3(n_observations, n_actions).to(device)
    load_model(agent, f"td3_{i}")

    if agent is not None:
        eval_scale(agent, episodes)
        print(f"{i} scale: {episodes}")
        i += 1

print("---")
print(f"scale: {episodes}")

0 scale: {1.0: [-46.26199999999026], 1.5: [9.208999999998532], 2.0: [46.14100000000284], 2.5: [81.18899999999132], 3.0: [95.42299999999166], 3.5: [96.07499999999308], 4.0: [95.81599999999266], 4.5: [59.16300000000002], 5.0: [42.33100000001287], 5.5: [26.155000000020546], 6.0: [37.65700000001437], 6.5: [63.94899999997307], 7.0: [80.91899999998493]}
1 scale: {1.0: [-46.26199999999026, 71.73299999998741], 1.5: [9.208999999998532, 58.73799999999155], 2.0: [46.14100000000284, 75.06799999999193], 2.5: [81.18899999999132, 91.8009999999888], 3.0: [95.42299999999166, 96.48099999999627], 3.5: [96.07499999999308, 97.99799999999632], 4.0: [95.81599999999266, 97.77499999999591], 4.5: [59.16300000000002, 97.52799999999552], 5.0: [42.33100000001287, 97.30099999999514], 5.5: [26.155000000020546, 97.07599999999475], 6.0: [37.65700000001437, 96.86099999999436], 6.5: [63.94899999997307, 96.64399999999395], 7.0: [80.91899999998493, 96.42899999999355]}
2 scale: {1.0: [-46.26199999999026, 71.73299999998741,

In [ ]:
def eval_starting_position(agent, episode_durations):
    agent.eval()
    agent.is_training = False

    max_episode_length = 500
    num_episodes = 100

    for extra_range in np.arange(0.0, 0.401, 0.05):

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()

            extra = np.random.uniform(-0.1 - extra_range, 0.1 + extra_range, env.starting_point.shape)
            #extra = np.random.uniform(0.1, 0.1 + extra_range, env.starting_point.shape)
            #extra = extra * (2*np.random.randint(0,2,size=env.starting_point.shape)-1)
            env.unwrapped.state = np.array(env.starting_point + extra, dtype=np.float32)
            env.unwrapped.state[2] = env.state[2] % (2 * math.pi)
            observation = env.normalised_state()

            # unsqueeze adds batch dimension
            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            episode_steps = 0
            done = False
            while not done:

                action = agent.select_action(state, False, False)
                action_i = action.detach().cpu().squeeze(0).numpy()

                observation, reward, done, info = env.step(action_i) 
                
                if max_episode_length and episode_steps >= max_episode_length - 1:
                    done = True
                
                episode_steps += 1 

                overall_reward += reward

                state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_durations[np.round(extra_range, 3)].append(overall_reward / num_episodes)

In [ ]:
episodes = {}
for extra_range in np.arange(0.0, 0.401, 0.05):
    episodes[np.round(extra_range, 3)] = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

env = NormalizedEnv(PointMazeEnv(4))

i = 0
while i < 10:
    agent = TD3(n_observations, n_actions).to(device)
    load_model(agent, f"td3_{i}")

    if agent is not None:
        eval_starting_position(agent, episodes)
        print(f"{i} extra_range: {episodes}")
        i += 1

print("---")
print(f"extra_range: {episodes}")

0 extra_range: {0.0: [95.8299999999922], 0.05: [95.78499999999227], 0.1: [96.04899999999284], 0.15: [95.89699999999267], 0.2: [92.87899999998905], 0.25: [88.58699999998639], 0.3: [84.1349999999902], 0.35: [81.15699999998394], 0.4: [74.01099999998468]}
1 extra_range: {0.0: [95.8299999999922, 97.76899999999588], 0.05: [95.78499999999227, 97.7669999999959], 0.1: [96.04899999999284, 91.83599999999355], 0.15: [95.89699999999267, 91.8379999999913], 0.2: [92.87899999998905, 90.35599999999647], 0.25: [88.58699999998639, 91.82399999999352], 0.3: [84.1349999999902, 74.02799999997728], 0.35: [81.15699999998394, 87.31999999998813], 0.4: [74.01099999998468, 88.77599999998944]}
2 extra_range: {0.0: [95.8299999999922, 97.76899999999588, 92.70199999998871], 0.05: [95.78499999999227, 97.7669999999959, 91.99799999998872], 0.1: [96.04899999999284, 91.83599999999355, 83.50999999998798], 0.15: [95.89699999999267, 91.8379999999913, 84.95799999999127], 0.2: [92.87899999998905, 90.35599999999647, 79.211999999

In [15]:
state_max = torch.from_numpy(env.observation_space.high).to(device).float()
state_min = torch.from_numpy(env.observation_space.low).to(device).float()
state_mid = (state_max + state_min) / 2.
state_range = (state_max - state_min)
def save_trajectories(agent, episode_durations, dirty):
    agent.eval()

    max_episode_length = 500
    agent.is_training = False

    num_episodes = 10

    l2norm = 0.3
    episode_durations.append([])
    
    for i_episode in range(num_episodes):
        path = {"overall_reward": 0, "worker": []}

        observation = env.reset()

        state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
        state_ = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_steps = 0
        overall_reward = 0
        done = False
        while not done:
            if dirty:
                state = state + state_range * torch.FloatTensor(state.shape).uniform_(-l2norm, l2norm).to(device)
                state = torch.max(torch.min(state, state_max), state_min).float()

            action = agent.select_action(state, False, False)
            action_i = action.detach().cpu().squeeze(0).numpy()
            path["worker"].append((episode_steps, state_.detach().cpu().squeeze(0).numpy(), action.detach().cpu().squeeze(0).numpy()))

            observation, reward, done, info = env.step(action_i) 
            
            if max_episode_length and episode_steps >= max_episode_length - 1:
                done = True
            
            episode_steps += 1 

            overall_reward += reward

            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
            state_ = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        path["overall_reward"] = overall_reward
        episode_durations[-1].append(path)

In [17]:
episodes = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

i = 0
while i < 10:
    #agent = train_model()
    agent = TD3(n_observations, n_actions).to(device)
    load_model(agent, f"td3_{i}")

    if agent is not None:
        # goal_attack, action_attack, same_noise
        save_trajectories(agent, episodes, True)
        #print(f"{i} paths: {episodes}")
        i += 1

print("----")
#print(f"paths: {episodes}")

torch.save(episodes, "td3_PointMaze_dirty_eps.pt")

----
